In [2]:
import pdfreader
from pdfreader import PDFDocument, SimplePDFViewer
import os
import re
import json

In [3]:
def open_pdf(path):
    fd = open(path, "rb")
    viewer = SimplePDFViewer(fd)
    doc = PDFDocument(fd)
    no_of_pages = len([p for p in doc.pages()])
    doc_text = ""
    for i in range(no_of_pages):
        viewer.navigate(i + 1)
        viewer.render()
        doc_text += "".join(viewer.canvas.strings)
    return doc_text, no_of_pages

In [4]:
REGEX_0 = " ;;;;; "   # simple placeholder

In [78]:
def correct_encoding(text):
    re_grp_1 = [
        (r"(\u2013)", "-"),
        (r"õ", "ı"),
        (r"Đ", "İ"),
        (r"_{2,}", " "),
        (r"\-{2,}", " "),
        (r"(\xa0)", " "),
        (r"(\u2010)", "-"),
        ("Türkiye Cumhuriyet Merkez Bankası İdare Merkezi İstiklal Caddesi 10 06100 Ulus / Ankara 0312 507 50 00 www.tcmb.gov.tr", ""),
        ("Türkiye Cumhuriyet Merkez Bankası İdare Merkezi Anafartalar Mah. İstiklal Cad. No:10 06050 Ulus Altındağ Ankara 0312 507 50 00 www.tcmb.gov.tr", ""),
        ("Türkiye Cumhuriyet Merkez Bankası İdare Merkezi Hacı Bayram Mah. İstiklal Cad. No:10 06050 Ulus Altındağ Ankara 0312 507 50 00 www.tcmb.gov.tr", ""),
        ("Türkiye Cumhuriyet Merkez Bankası İdare Merkezi Hacı Bayram Mah. İstiklal Cad. No:10 06050  Ulus Altındağ Ankara 0312 507 50 00  www.tcmb.gov.tr", "")
    ]

    re_grp_2 = [
        (r"([a-zğüşıöçâîû])\x01|\x01([a-zğüşıöçâîû])", "\\1ş\\2"),
        (r"([A-ZĞÜŞİÖÇÂÎÛ])\x01|\x01([A-ZĞÜŞİÖÇÂÎÛ])", "\\1İ\\2"),
        (r"(\x02)", "ğ"),
        (r"(\x03)", "İ")
    ]

    re_grp_3 = [
        (r"(\x01)", "Ş"),
        (r"([a-zğüşıöçâîû])\x02|\x02([a-zğüşıöçâîû])", "\\1ş\\2"),
        (r"([A-ZĞÜŞİÖÇÂÎÛ])\x02|\x02([A-ZĞÜŞİÖÇÂÎÛ])", "\\1İ\\2"),
        (r"(\x03)", "İ"),
        (r"(\x04)", "ğ")
    ]

    re_grp_4 = [
        (r"([a-zğüşıöçâîû])\x01|\x01([a-zğüşıöçâîû])", "\\1ğ\\2"),
        (r"([A-ZĞÜŞİÖÇÂÎÛ])\x01|\x01([A-ZĞÜŞİÖÇÂÎÛ])", "\\1İ\\2"),
        (r"(\x02)", "ş"),
        (r"(\x03)", "İ")
    ]

    re_grp_5 = [
        (r"([a-zğüşıöçâîû])\x01|\x01([a-zğüşıöçâîû])", "\\1ş\\2"),
        (r"([A-ZĞÜŞİÖÇÂÎÛ])\x01|\x01([A-ZĞÜŞİÖÇÂÎÛ])", "\\1İ\\2"),
        (r"(\x02)", "İ"),
        (r"(\x03)", "ğ")
    ]

    for regex in re_grp_1:
        text = re.sub(regex[0], regex[1], text)

    if len(re.findall(r"(\x04)", text)) > 0:
        for regex in re_grp_3:
            text = re.sub(regex[0], regex[1], text)
    else:
        if text.startswith(" 1Sayı:2007-32"):
            for regex in re_grp_4:
                text = re.sub(regex[0], regex[1], text)
        elif text.startswith("Sayı:2007-24") or text.startswith("Sayı:2007-08"):
            for regex in re_grp_5:
                text = re.sub(regex[0], regex[1], text)
        else:
            for regex in re_grp_2:
                text = re.sub(regex[0], regex[1], text)

    return text

In [6]:
def discard_metadata(text, doc_name=None):
    regex3 = r"((Sayı|No)\s?:?\s?(\d{4}\s?\-\s?)?\d{1,2})"
    regex4 = r"((Para Politikası Kurulu Toplantı Özeti)|(PARA POLİTİKASI KURULU (TOPLANTI|DEĞERLENDİRMELERİ) ÖZETİ))"
    regex5 = r"(Toplantı Tarih(ler)?i: (\d{1,2} ve )?\d{1,2} [a-zA-ZşığüŞİĞÜ]{4,7} \d{4})"
    regex6 = r"(\d{1,2}\s{,2}[a-zA-ZşığüŞİĞÜ]{4,7}\s\d{4})"
    regex7 = r"BASIN DUYURUSU"

    res_0 = re.findall(regex3, text)
    res_1 = re.findall(regex4, text)
    res_2 = re.findall(regex5, text)
    res_4 = re.findall(regex7, text)
    # print(res_0)
    # print(res_1)
    # print(res_2)
    # print(res_4)

    text = re.sub(regex3, " ", text)
    text = re.sub(regex4, " ", text)
    text = re.sub(regex5, " ", text)

    res_3 = re.findall(regex6, text)
    # print(res_3[0])
    
    text = re.sub(regex6, " ", text, 1)
    text = re.sub(regex7, " ", text)

    if len(res_0) > 0:
        number = res_0[0][0].split(":")[1].replace(" ", "") if ":" in res_0[0][0] else res_0[0][0].replace("Sayı", "").replace(" ", "")
    else:
        number = doc_name.split(".")[0].replace("DUY", "")
    date = " ".join([r for r in res_2[0][0].split(":")[1].strip().split(" ") if r != ""])
    published = " ".join([r for r in res_3[0].strip().split(" ") if r != ""]) if len(res_3) > 0 else ""
    
    return text, (number, date, published)

In [115]:
def discard_page_numbers(text, no_of_pages, doc_name=None):
    re_1 = r"(^\s?1)"
    re_2 = r"((^\s?1)|(\s([2-9]|[1-9][0-9]))\s\s)"
    re_3 = r"((^\s?1\s\s)|((\s([2-9]|[1-9][0-9])\s\s)|(\s\s([2-9]|[1-9][0-9])\s)))"
    re_4 = r"((\s([1-9]|[1-9][0-9])\s\s)|(\s\s([2-9]|[1-9][0-9])\s)|(\s" + str(no_of_pages) + "\s$))"
    result = re.findall(re_2, text)
    test_numbers = "".join([r[0].strip()for r in result])
    test_truth = "".join([str(i + 1) for i in range(no_of_pages)])
    # print(result)
    if test_truth != test_numbers:
        result = re.findall(re_3, text)
        test_numbers = "".join([r[0].strip()for r in result])
        re_2 = re_3
        # print(result)
    if test_truth != test_numbers:
        result = re.findall(re_4, text)
        test_numbers = "".join([r[0].strip()for r in result])
        re_2 = re_4
        # print(result)

    if test_truth == test_numbers:
        print(doc_name, "got page numbers, matched")
        text = re.sub(re_2, REGEX_0, text)
    elif len(re.findall(re_1, text)) > 0:
        print(doc_name, "got page numbers, not matched")
        text = re.sub(r"(^\s?1)", "", text)
        for page in range(1, no_of_pages):
            res_seq = re.findall("(  %s)" % str(page + 1), text)
            print(res_seq)
            text = re.sub("(  %s)" % str(page + 1), REGEX_0, text, 1)

    # print(doc_name, test_truth, test_numbers)

    return text

In [114]:
def discard_references(text):
    re_1 = r"(([a-zğüşıöçâîû])(\.\d|\d\.)\s)"
    res_1 = re.findall(re_1, text)
    print(res_1)
    text = re.sub(re_1, "\\2. ", text)

    # discard footnotes

    if len(res_1) > 0:    
        # re_0 = r"(\s{2,}\d\s[^;]+(?=" + REGEX_0.strip() + "))"
        # res_0 = re.findall(re_0, text)
        # # print(res_0)
        # text = re.sub(re_0, "", text)

        re_0 = r"((\s){2,}[1-9][0-9]?\s[a-zA-Z0-9ğüşıöçĞÜŞİÖÇ:,\s\.“”]{50,}(\s([2-9]|[1-9][0-9])\s\s))"
        text = re.sub(re_0, "\\4", text)
    
    text = text.replace(REGEX_0.strip(), "")
    return text

In [62]:
def parse_document(text):
    regex1 = r"(\d{1,2}\.\s{1,2}(?=[A-ZĞÜŞİÖÇÂÎÛ\d]))"
    regex2 = r"((^|\.)\s+([A-ZĞÜŞİÖÇÂÎÛ\d][a-zğüşıöçâîû\d]+\s{1,2})+(?=[a-zğüşıöçâîû]))"
    regex3 = r"([a-zA-Z0-9ğüşıöçĞÜŞİÖÇÂâÎîÛû:,;\-\*\+/'’’“”\"\(\)\s]{2,}(\.|:))"
    regex4 = r"(^(Toplantıya Katılan Kurul Üyeleri)([a-zA-Z0-9ğüşıöçĞÜŞİÖÇÂâÎîÛû,\s\[\]]+)(\.|(\([^\)]+\))))"
    res = re.findall(regex1, text)
    
    print(res)

    topics = [] 
    
    if len(res) > 0:
        text = re.sub(regex1, REGEX_0 + "\\1", text)
        parts = text.split(REGEX_0)

        topic = re.sub(r"(^,)", "", re.sub(regex4, "", parts[0].strip())).strip()     # in case of DUY2009-48.pdf and DUY2018-06.pdf
        topic_items = []

        for i in range(1, len(parts)):
            p = parts[i]
            p = re.sub(regex1, "", p)
            res2 = re.findall(regex3, p)
            p = re.sub(regex3, "", p)
            # print(p)
            item_idx = int(res[i - 1].split(".")[0])
            item_text = " ".join([r[0].strip() for r in res2])
            item_text = re.sub(r"\s{2,}", " ", item_text)
            topic_items.append({"index": item_idx, "text": item_text})

            if p.strip() == "":
                if i == len(parts) - 1:
                    topics.append({"title": topic, "items": topic_items})
            else:
                topics.append({"title": topic, "items": topic_items})
                topic = p.strip()
                topic_items = []
            
            
    else:
        res2 = re.findall(regex2, text)
        print(res2)
        temp1 = [re.sub(r"\s{2,}", " ", r[0].replace(".", "").strip()) for r in res2]
        temp2 = [" ".join(r.split(" ")[0:-1]) for r in temp1 if len(r.split(" ")) > 1]
        temp3 = []
        
        for r in temp2:
            r_p = r.split(" ")
            if len(r_p) % 2 == 1:
                temp3.append(r)
                continue
            no_dup = False
            for j in range(int(len(r_p)/2)):
                if r_p[j] != r_p[j + int(len(r_p)/2)]:
                    no_dup = True
                    break
            if no_dup:
                temp3.append(r)
                continue
            r_n = " ".join([r_p[j] for j in range(int(len(r_p)/2))])
            temp3.append(r_n)
            
        print(len(temp3), temp3)

        for r in res2:
            simplified = re.sub(r"\s{2,}", " ", r[0].replace(".", "").strip())
            if len(simplified.split(" ")) > 1:
                text = re.sub(r[0], r[1] + REGEX_0 + simplified + " ", text, 1)
            
        parts = text.split(REGEX_0)
        print(len(parts), parts)
        for i, r in enumerate(parts):
            if i == 0:
                continue
            topics.append({"title": temp3[i - 1], "items": [{"index": 0, "text": re.sub(r"\s{2,}", " ", re.sub(temp3[i - 1], " ", r, 1)).strip()}]})
        

    print([t["title"] for t in topics])

    return topics

In [79]:
def minor_changes(text, reverse=False):
    changes = [
        ("Beklenti Anketi", "Beklenti anketi"),
        ("Riskler ve Para Politikası", "Riskler Ve Para Politikası"),
        ("İktisadi Yönelim Anketi", "İktisadi yönelim anketi"),
        ("Para Politikası Kurulu,", "Para Politikası Kurulu"),
        ("Para Politikası Kurulu 23 Şubat", "Para Politikası Kurulu, 23 Şubat"),
        ("M. İbrahim Turhan", "İbrahim Mustafa Turhan"),
        ("(Başkan)", "[Başkan]"),
        ("tutulmasınakarar", "tutulmasına karar"),
        ("Ocak-Nisan 2006 10.", "10. Ocak-Nisan 2006:"),
        ("Mayıs 2006 ve Sonrası 11.", "11. Mayıs 2006 ve Sonrası:"),
        ("Mehmet Yörükoğlu  Para Politikası", "Mehmet Yörükoğlu.  Para Politikası"),
        ("Mehmet Yörükoğlu      Enflasyon Gelişmeleri", "Mehmet Yörükoğlu.      Enflasyon Gelişmeleri"),
        ("İbrahim Mustafa Turhan Abdullah Yavaş", "İbrahim Mustafa Turhan, Abdullah Yavaş"),
        ("Mehmet Yörükoğlu Katılmayan", "Mehmet Yörükoğlu. Katılmayan"),
        ("Mehmet Yörükoğlu  Toplantıya", "Mehmet Yörükoğlu.  Toplantıya"),
        ("6. Özetle, enflasyonda Ağustos ayından itibaren düşüşün başlayacağı öngörülmekle birlikte, fiyatlama davranışları dikkatle takip edilmektedir", "6. Özetle, enflasyonda Ağustos ayından itibaren düşüşün başlayacağı öngörülmekle birlikte, fiyatlama davranışları dikkatle takip edilmektedir."),
        ("Yatırım eğiliminin düşük seviyelerde seyretmesi ve sanayi üretimindeki ılımlı artış eğilimi istihdam piyasasındaki iyileşmenin zaman alabileceğine işaret etmektedir", "Yatırım eğiliminin düşük seviyelerde seyretmesi ve sanayi üretimindeki ılımlı artış eğilimi istihdam piyasasındaki iyileşmenin zaman alabileceğine işaret etmektedir."),
        ("Toplantıya Katılan Kurul ÜyeleriToplantıya Katılan Kurul ÜyeleriToplantıya Katılan Kurul ÜyeleriToplantıya Katılan Kurul Üyeleri", "Toplantıya Katılan Kurul Üyeleri")
    ]
    for ch in changes:
        text = text.replace(ch[0], ch[1]) if not reverse else text.replace(ch[1], ch[0])

    return text

In [76]:
def process_document(text, no_of_pages, doc_name):
    text = correct_encoding(text)
    text, meta = discard_metadata(text, doc_name)
    print(meta)
    text = discard_page_numbers(text, no_of_pages, doc_name)
    text = discard_references(text)
    text = minor_changes(text)
    topics = parse_document(text)
    
    return {
        "number": meta[0],
        "date": meta[1],
        "published": meta[2],
        "topics": topics
    }

In [51]:
def process_brief(text, no_of_pages, doc_name=None):
    text = correct_encoding(text)
    text = minor_changes(text)
    
    print(text)

    re_0 = r"((^\s?1)|(\s[2-9])\s\s)"
    result = re.findall(re_0, text)
    
    print(no_of_pages, len(result), doc_name)

    text = re.sub(re_0, "", text)

    regex1 = "[Başkan]"
    regex3 = r"((Sayı|No)\s?:?\s?(\d{4}\s?\-\s?)?\d{1,2})"
    regex4 = r"((Para Politikası Kurulu Kararı)|(PARA POLİTİKASI KURULU KARARI))" 
    regex6 = r"(\d{1,2}\s{1,2}[a-zA-ZşığüŞİĞÜ]{4,7}\s\d{4})"
    regex7 = r"BASIN DUYURUSU"
    regex8 = r"((Toplantıya Katılan Kurul Üyeleri)([a-zA-Z0-9ğüşıöçĞÜŞİÖÇÂâÎîÛû,\s\[\]]+)(\.|(\([^\)]+\))))"
    regex9 = r"(^\([^\)]+\))"
    regex10 = r"(^((Toplantıya )?Katıla?mayan Kurul Üyesi:?)([a-zA-Z0-9ğüşıöçĞÜŞİÖÇÂâÎîÛû,\s\[\]]+)(\([^\)]+\)))"

    res = re.findall(regex8, text)
    print(res)
    new_text = re.sub(regex8, REGEX_0 + "\\1", text)
    new_text_parts = new_text.split(REGEX_0)
    parsed = {
        "number": "",
        "text": "",
        "date": "",
        "governor": "",
        "board": []
    }
    ress_0 = re.findall(regex4, new_text_parts[0])
    print(ress_0)
    new_text_parts[0] = re.sub(regex4, "", new_text_parts[0])

    ress_1 = re.findall(regex7, new_text_parts[0])
    print(ress_1)
    new_text_parts[0] = re.sub(regex7, "", new_text_parts[0])

    if doc_name == "DUY2018-01.pdf":
        parsed["number"] = "2018-01"
        parsed["date"] = "18 Ocak 2018"
    elif doc_name == "DUY2018-10.pdf":
        parsed["number"] = "2018-10"
        parsed["date"] = "25 Nisan 2018"
    else:
        res_0_0 = re.findall(regex3, new_text_parts[0])
        if len(res_0_0) > 0:
            print(res_0_0)
            parsed["number"] = res_0_0[0][0].split(":")[1].replace(" ", "") if ":" in res_0_0[0][0] else res_0_0[0][0].replace("Sayı", "").replace(" ", "")
            new_text_parts[0] = re.sub(regex3, "", new_text_parts[0])
        else:
            res_1_0 = re.findall(regex3, new_text_parts[1])
            print(res_1_0)
            if len(res_1_0) > 0:
                parsed["number"] = res_1_0[0][0].split(":")[1].replace(" ", "") if ":" in res_1_0[0][0] else res_1_0[0][0].replace("Sayı", "").replace(" ", "")
                new_text_parts[1] = re.sub(regex3, "", new_text_parts[1])
                new_text_parts[1] = re.sub(regex6, "", new_text_parts[1])
            else:
                parsed["number"] = doc_name.split(".")[0].replace("DUY", "")
    

        res_0_1 = re.findall(regex6, new_text_parts[0])
        print(res_0_1)
        parsed["date"] = " ".join([r for r in res_0_1[0].strip().split(" ") if r != ""])
    
    
    new_text_parts[1] = re.sub(regex8, "", new_text_parts[1])
    members = res[0][2]
    remainder = re.sub(r"(^\.)", "", re.sub(regex10, "", re.sub(regex9, "", new_text_parts[1].strip())))   # in case of karar/2010/1.pdf karar/2010/1.pdf karar/2010/2.pdf karar/2010/5.pdf
    parsed["text"] = re.sub(r"\s{2,}", " ", remainder).strip()
    parsed["governor"] = [m.replace(regex1, "").strip() for m in members.split(",") if regex1 in m][0]
    parsed["board"] = [m.strip() for m in members.split(",") if regex1 not in m]
    print(parsed["governor"], parsed["board"])
    print(parsed["text"])

    return parsed

In [13]:
def export_json(path, file, text):
    output_path = path.replace(FOLDER_NAME, OUTPUT_FOLDER).replace(PATH_EXT_1, "")
    if not os.path.isdir(output_path):
        os.makedirs(output_path)
    with open(os.path.join(output_path, file.replace("pdf", "json")), "w", encoding="utf-8") as f:
        json.dump(text, f, ensure_ascii=False, indent=4)

In [120]:
FOLDER_NAME = "ppk"
OUTPUT_FOLDER = "ppk_json"
PATH_EXT_0 = "karar"
PATH_EXT_1 = "özet"
PATH_YEAR = "2024"

In [ ]:
pdf_data = []
for path, dirs, files in os.walk(f"{FOLDER_NAME}/{PATH_EXT_1}/{PATH_YEAR}"):
    for file in files:
        if not file.endswith(".pdf"):
            continue
        print(file)
        doc_text, no_of_pages = open_pdf(os.path.join(path, file))
        processed = process_document(doc_text, no_of_pages, file)
        pdf_data.append(processed)

In [121]:
brief_data = []
for path, dirs, files in os.walk(f"{FOLDER_NAME}/{PATH_EXT_0}/{PATH_YEAR}"):
    for file in files:
        if not file.endswith(".pdf"):
            continue
        print("\n\n")
        doc_text, no_of_pages = open_pdf(os.path.join(path, file))
        processed_text = process_brief(doc_text, no_of_pages, file)
        brief_data.append(processed_text)




1  Para Politikası Kurulu Kararı 22 Şubat 2024 Sayı: 2024-09   Toplantıya Katılan Kurul Üyeleri Yaşar Fatih Karahan [Başkan], Osman Cevdet Akçay, Elif Haykır Hobikoğlu, Hatice Karahan, Fatma Özkul. Para Politikası Kurulu (Kurul) politika faizi olan bir hafta vadeli repo ihale faiz oranının yüzde 45 düzeyinde sabit tutulmasına karar vermiştir. Ocak ayında aylık enflasyonun ana eğilimi yılın ilk ayına özgü zamana-bağlı fiyat ve ücret güncellemeleri sonucunda, enflasyon tahmin patikasında öngörülen şekilde yükselmiş ve manşet enflasyon sınırlı bir artış kaydetmiştir. Yakın döneme ilişkin göstergeler, yurt içi talepteki dengelenmenin sürdüğüne işaret etmektedir. Söz konusu dengelenme süreci, tüketim malı ve altın ithalatında güçlü, diğer tüketim harcamalarına ilişkin göstergelerde ise öngörülene kıyasla yavaş seyretmektedir. Buna ek olarak, hizmet fiyatlarındaki katılık, jeopolitik riskler ve gıda fiyatları enflasyon baskılarını canlı tutmaktadır. Kurul, enflasyon beklentileri ve fiyatl

In [122]:
pdf_data = []
for path, dirs, files in os.walk(f"{FOLDER_NAME}/{PATH_EXT_1}/{PATH_YEAR}"):
    for file in files:
        if not file.endswith(".pdf"):
            continue
        # if file in ["DUY2006-08.pdf", "DUY2006-11.pdf", "DUY2006-20.pdf", "DUY2006-29.pdf", "DUY2006-34.pdf"]:
        #     continue
        print("\n",file)
        doc_text, no_of_pages = open_pdf(os.path.join(path, file))
        processed_text = process_document(doc_text, no_of_pages, file)
        related_decision = [data for data in brief_data if data["date"].endswith(processed_text["date"]) or processed_text["date"].endswith(data["date"])][0]
        processed_text["decision"] = related_decision["number"]
        processed_text["governor"] = related_decision["governor"]
        processed_text["board"] = related_decision["board"]
        processed_text["abstract"] = related_decision["text"]
        output_path = path.replace(FOLDER_NAME, OUTPUT_FOLDER).replace(PATH_EXT_1, "")
        if not os.path.isdir(output_path):
            os.makedirs(output_path)
        with open(os.path.join(output_path, file.replace("pdf", "json")), "w", encoding="utf-8") as f:
            json.dump(processed_text, f, ensure_ascii=False, indent=4)
        pdf_data.append(processed_text)


 DUY2024-41.pdf
('2024-41', '23 Temmuz 2024', '30 Temmuz 2024')
DUY2024-41.pdf got page numbers, matched
[]
['1. ', '2. ', '3. ', '4. ', '5. ', '6. ', '7. ', '8. ', '9. ', '10. ', '11. ', '12. ', '13. ', '14. ', '15. ', '16. ', '17. ', '18. ', '19. ', '20. ', '21. ', '22. ', '23. ', '24. ', '25. ', '26. ', '27. ', '28. ', '29. ', '30. ']
['Küresel Ekonomi', 'Parasal ve Finansal Koşullar', 'Talep ve Üretim', 'Enflasyon Gelişmeleri ve Beklentiler', 'Para Politikası']

 DUY2024-36.pdf
('2024-36', '27 Haziran 2024', '3 Temmuz 2024')
DUY2024-36.pdf got page numbers, matched
[]
['1. ', '2. ', '3. ', '4. ', '5. ', '6. ', '7. ', '8. ', '9. ', '10. ', '11. ', '12. ', '13. ', '14. ', '15. ', '16. ', '17. ', '18. ', '19. ', '20. ', '21. ', '22. ', '23. ', '24. ', '25. ', '26. ', '27. ', '28. ', '29. ', '30. ', '31. ']
['Küresel Ekonomi', 'Parasal ve Finansal Koşullar', 'Talep ve Üretim', 'Enflasyon Gelişmeleri', 'Para Politikası']

 DUY2024-25.pdf
('2024-25', '25 Nisan 2024', '3 Mayıs 2024')
DUY

In [111]:
_doc_text, _no_of_pages = open_pdf(os.path.join("ppk/özet/2023", "DUY2023-29.pdf"))

In [43]:
_doc_text, _no_of_pages = open_pdf(os.path.join("ppk/karar/2018", "DUY2018-01.pdf"))

In [116]:
_doc_text

' 1   Para Politikası Kurulu Toplantı Özeti 27 Temmuz 2023, Sayı: 2023-29    Toplantı Tarihi: 20 Temmuz 2023 Küresel Ekonomi 1. Küresel enflasyon düşerken, halen uzun dönem ortalamalarının ve merkez bankalarının hedeflerinin üzerinde seyretmektedir. Gelişmiş ve gelişmekte olan ekonomilerin tüketici enflasyonları bir önceki PPK dönemine göre yüzde 5,59 ve yüzde 5,82 düzeylerinden sırasıyla yüzde 4,91 ve yüzde 5,41 seviyelerine gerilemiştir. Son 10 yıllık dönemde ise ortalama enflasyon gelişmiş ekonomilerde yüzde 2,2 ve gelişmekte olan ekonomilerde yüzde 5,6 seviyesinde gerçekleşmiştir. Enflasyon, gelişmiş ülkelerde yüzde 2; gelişmekte olan ülkelerde ise ortalama yüzde 3,5 olan hedef oranların belirgin olarak üzerinde seyretmeye devam etmektedir. 2023 yılı son çeyrek ortalama yıllık enflasyonun gelişmiş ekonomilerde 3,1, gelişmekte olan ülkelerde ise yüzde 5,9 seviyesinde gerçekleşmesi beklenmektedir. Emtia fiyatlarında geçen yıl ortasından itibaren başlayan gerileme eğilimi sürmekle bir

In [119]:
print(discard_page_numbers(_doc_text, _no_of_pages))

None got page numbers, not matched
['  2', '  2']
['  3', '  3', '  3', '  3', '  3']
['  4', '  4', '  4', '  4', '  4', '  4']
['  5']
['  6', '  6']
['  7']
['  8']
   Para Politikası Kurulu Toplantı Özeti 27 Temmuz 2023, Sayı: 2023-29    Toplantı Tarihi: 20 Temmuz 2023 Küresel Ekonomi 1. Küresel enflasyon düşerken, halen uzun dönem ortalamalarının ve merkez bankalarının hedeflerinin üzerinde seyretmektedir. Gelişmiş ve gelişmekte olan ekonomilerin tüketici enflasyonları bir önceki PPK dönemine göre yüzde 5,59 ve yüzde 5,82 düzeylerinden sırasıyla yüzde 4,91 ve yüzde 5,41 seviyelerine gerilemiştir. Son 10 yıllık dönemde ise ortalama enflasyon gelişmiş ekonomilerde yüzde 2,2 ve gelişmekte olan ekonomilerde yüzde 5,6 seviyesinde gerçekleşmiştir. Enflasyon, gelişmiş ülkelerde yüzde 2; gelişmekte olan ülkelerde ise ortalama yüzde 3,5 olan hedef oranların belirgin olarak üzerinde seyretmeye devam etmektedir. 2023 yılı son çeyrek ortalama yıllık enflasyonun gelişmiş ekonomilerde 3,1, geli

In [117]:
_processed_text = correct_encoding(_doc_text)
_processed_text, meta = discard_metadata(_processed_text)
_processed_text = discard_page_numbers(_processed_text, _no_of_pages)
_processed_text

None got page numbers, not matched
['  2', '  2']
['  3', '  3', '  3', '  3', '  3']
['  4', '  4', '  4', '  4', '  4', '  4']
['  5']
['  6', '  6']
['  7']
['  8']


'      ,        Küresel Ekonomi 1. Küresel enflasyon düşerken, halen uzun dönem ortalamalarının ve merkez bankalarının hedeflerinin üzerinde seyretmektedir. Gelişmiş ve gelişmekte olan ekonomilerin tüketici enflasyonları bir önceki PPK dönemine göre yüzde 5,59 ve yüzde 5,82 düzeylerinden sırasıyla yüzde 4,91 ve yüzde 5,41 seviyelerine gerilemiştir. Son 10 yıllık dönemde ise ortalama enflasyon gelişmiş ekonomilerde yüzde 2,2 ve gelişmekte olan ekonomilerde yüzde 5,6 seviyesinde gerçekleşmiştir. Enflasyon, gelişmiş ülkelerde yüzde 2; gelişmekte olan ülkelerde ise ortalama yüzde 3,5 olan hedef oranların belirgin olarak üzerinde seyretmeye devam etmektedir. 2023 yılı son çeyrek ortalama yıllık enflasyonun gelişmiş ekonomilerde 3,1, gelişmekte olan ülkelerde ise yüzde 5,9 seviyesinde gerçekleşmesi beklenmektedir. Emtia fiyatlarında geçen yıl ortasından itibaren başlayan gerileme eğilimi sürmekle birlikte, Emtia Fiyat Endeksinin mevcut seviyesi son on yılın ortalamasının yüzde 26,2 üzerinded

In [110]:
_processed_text = discard_references(_processed_text)
_processed_text

[('r.1 ', 'r', '.1')]


'      ,        Küresel Ekonomi 1. Küresel enflasyon düşerken, halen uzun dönem ortalamalarının ve merkez bankalarının hedeflerinin üzerinde seyretmektedir. Gelişmiş ve gelişmekte olan ekonomilerin tüketici enflasyonları bir önceki PPK dönemine göre yüzde 5,59 ve yüzde 5,82 düzeylerinden sırasıyla yüzde 4,91 ve yüzde 5,41 seviyelerine gerilemiştir. Son 10 yıllık dönemde ise ortalama enflasyon gelişmiş ekonomilerde yüzde 2,2 ve gelişmekte olan ekonomilerde yüzde 5,6 seviyesinde gerçekleşmiştir. Enflasyon, gelişmiş ülkelerde yüzde 2; gelişmekte olan ülkelerde ise ortalama yüzde 3,5 olan hedef oranların belirgin olarak üzerinde seyretmeye devam etmektedir. 2023 yılı son çeyrek ortalama yıllık enflasyonun gelişmiş ekonomilerde 3,1, gelişmekte olan ülkelerde ise yüzde 5,9 seviyesinde gerçekleşmesi beklenmektedir. Emtia fiyatlarında geçen yıl ortasından itibaren başlayan gerileme eğilimi sürmekle birlikte, Emtia Fiyat Endeksinin mevcut seviyesi son on yılın ortalamasının yüzde 26,2 üzerinded

In [23]:
regex1 = r"(\d{1,2}\.\s(?=[A-ZĞÜŞİÖÇÂÎÛ\d]))"
regex2 = r"((^|\.)\s+([A-ZĞÜŞİÖÇÂÎÛ\d][a-zğüşıöçâîû\d]+\s{1,2})+(?=[a-zğüşıöçâîû]))"
res_0 = re.findall(regex1, _processed_text)
print(res_0)
# regex5 = r"(Toplantı Tarih(ler)?i: \d\d? [a-zA-ZşığüŞİĞÜ]{4,7} \d\d\d\d)" 
# res_1 = re.findall(regex5, text)
# print(res_1)

['1. ', '2. ', '3. ', '4. ', '5. ', '6. ', '7. ', '8. ', '9. ', '10. ', '11. ', '12. ', '14. ', '15. ', '16. ', '17. ', '18. ', '19. ', '20. ', '21. ']


In [112]:
temp2

['Enflasyon Gelişmeleri', 'Enflasyonu Etkileyen', 'Para Politikası']

In [211]:
_processed_text = process_document(_doc_text, _no_of_pages, "file")

('2006-52', '20 ve 25 Haziran 2006', '28 Haziran 2006')
['1. ', '2. ', '3. ', '4. ', '5. ', '6. ', '7. ', '8. ', '9. ', '10. ', '11. ', '12. ', '13. ', '14. ', '15. ', '16. ', '17. ', '18. ', '19. ', '20. ', '21. ', '22. ', '23. ', '24. ']
['Enflasyon Gelişmeleri ve Kısa Vadeli Tahminler', 'Enflasyonu Etkileyen Unsurlar', '20 Haziran ve 25 Haziran Para Politikası Kararları', 'Görünüm', 'Riskler ve Politika Tepkisi']


In [224]:
_processed_text = process_brief(_doc_text, _no_of_pages, "file")

Sayı:2007-6                                                                                   15 Şubat 2007 PARA POLİTİKASI KURULU KARARI Toplantı Tarihi: 15 Şubat 2007  Toplantıya Katılan Kurul Üyeleri Durmuş Yılmaz [Başkan], Erdem Başçı, Burhan Göklemez, İbrahim Mustafa Turhan, Mehmet Yörükoğlu.  Para Politikası Kurulu (Kurul), Bankamız bünyesindeki Bankalararası Para Piyasası ve İstanbul Menkul Kıymetler Borsası Repo-Ters Repo Pazarı’nda uygulanmakta olan kısa vadeli faiz oranlarının aşağıdaki gibi sabit tutulmasına karar vermiştir: a) Gecelik faiz oranları: Merkez Bankası borçlanma faiz oranı yüzde 17,50, borç verme faiz oranı yüzde 22,50, b) Geç Likidite Penceresi Faiz Oranları: Geç Likidite Penceresi uygulaması çerçevesinde, Bankalararası Para Piyasası’nda saat 16.00-16.30 arası gecelik vadede uygulanan Merkez Bankası borçlanma faiz oranı yüzde 13,50, borç verme faiz oranı yüzde 25,50, c) Açık piyasa işlemleri çerçevesinde piyasa yapıcısı bankalara repo işlemleri yoluyla gecelik 

In [212]:
_processed_text

{'number': '2006-52',
 'date': '20 ve 25 Haziran 2006',
 'published': '28 Haziran 2006',
 'topics': [{'title': 'Enflasyon Gelişmeleri ve Kısa Vadeli Tahminler',
   'items': [{'index': 1,
     'text': 'Yılın ilk beş ayında enflasyondaki yükseliş büyük oranda arz yönlü unsurlardan kaynaklanmıştır. İşlenmemiş gıda ürünleri grubu fiyatları 2005 yılının ilk beş aylık döneminde birikimli olarak hemen hiç değişmezken, 2006 yılının aynı döneminde yüzde 11,4 oranında artmıştır. Bunun yanında, döviz kuru hareketlerinin Mayıs ayı enflasyonuna etkisi yaklaşık 0,65 puan olmuştur. Söz konusu etkiler hariç tutulduğunda, ilk beş aylık dönemdeki fiyat artış oranının geçtiğimiz yılın aynı dönemindeki orana yakın olduğu görülmektedir. Bu gözlem, yılın ilk beş ayında enflasyonun ana eğiliminde belirgin bir değişiklik olmadığına işaret etmektedir.'},
    {'index': 2,
     'text': 'İşlenmemiş gıda ürünleri fiyatlarının Haziran ve Temmuz aylarında enflasyona olumlu katkı yapma olasılığı yüksektir. Buna karşı

In [ ]:
FOLDER_NAME = "ppk"
pdf_data = []
for path, dirs, files in os.walk(f"{FOLDER_NAME}/özet/2024"):
    for file in files:
        if file.endswith(".pdf"):
            doc_text, no_of_pages = open_pdf(os.path.join(path, file))
            processed_text = process_doc(doc_text, no_of_pages, file)
            pdf_data.append({
                "text_raw": doc_text,
                "text": processed_text,
                "pages": no_of_pages,
                "name": file
            })

In [ ]:
regex1 = r"([1-9][0-9]?\.\s)"
regex2 = ":::(()):::"
regex3 = r"(Sayı: \d\d\d\d-\d\d)"
regex4 = r"((Para Politikası Kurulu Toplantı Özeti)|(PARA POLİTİKASI KURULU TOPLANTI ÖZETİ))"
regex5 = r"(Toplantı Tarihi: \d\d ([A-Z]|[a-z]|ş|ı|ğ|ü|Ş|İ|Ğ|Ü){4,7} \d\d\d\d)"
regex6 = r"(\d\d ([A-Z]|[a-z]|ş|ı|ğ|ü|Ş|İ|Ğ|Ü){4,7} \d\d\d\d)"
regex7 = r"BASIN DUYURUSU"
regex8 = r"(([a-z]|[A-Z]|[0-9]|ğ|ü|ş|ı|ö|ç|Ğ|Ü|Ş|İ|Ö|Ç|Â|â|Î|î|Û|û|:|,|;|-|\s|’|“|”|\(|\)){2,}\.)"

res = re.findall(regex1, pdf_data[0]['text'])
new_text = re.sub(regex1, regex2 + "\\1", pdf_data[0]['text'])
new_text_parts = new_text.split(regex2)
parsed = {
    "number": "",
    "date": "",
    "published": "",
    "subjects": []
}
paragraph = ""
paragraph_items = []
for i, p in enumerate(new_text_parts):
    res2 = re.findall(regex1, p)
    if len(res2) == 0:
        res_0_0 = re.findall(regex3, p)
        res_0_1 = re.findall(regex4, p)
        res_0_2 = re.findall(regex5, p)
        res_0_3 = re.findall(regex7, p)
        parsed["number"] = res_0_0[0].split(":")[1].strip()
        parsed["date"] = res_0_2[0][0].split(":")[1].strip()
        p = re.sub(regex3, "", p)
        p = re.sub(regex4, "", p)
        p = re.sub(regex5, "", p)
        p = re.sub(regex7, "", p)
        res_0_3 = re.findall(regex6, p)
        parsed["published"] = res_0_3[0][0]
        p = re.sub(regex6, "", p)
        p = p.replace(",", "")
        paragraph = p.strip()
        
    elif len(res2) == 1:
        p = re.sub(regex1, "", p)
        res3 = re.findall(regex8, p)
        p = re.sub(regex8, "", p)
        item_idx = int(res2[0].split(".")[0])
        item_text = "".join([r[0].strip() for r in res3])
        paragraph_items.append({"index": item_idx, "text": item_text})
        if p.strip() == "":
            if i == len(new_text_parts) - 1:
                parsed["subjects"].append({"title": paragraph, "items": paragraph_items})
        else:
            parsed["subjects"].append({"title": paragraph, "items": paragraph_items})
            paragraph = p.strip()
            paragraph_items = []
            
    else:
        print("error")

In [ ]:
parsed

In [ ]:
for i, d in enumerate(pdf_data):

    # discard footnotes
    
    re_0 = r"((\s){2,}[1-9][0-9]?\s([a-z]|[A-Z]|[0-9]|ğ|ü|ş|ı|ö|ç|Ğ|Ü|Ş|İ|Ö|Ç|:|,|\s|\.|“|”){50,}(\s([2-9]|[1-9][0-9])\s\s))"
    d["doc_text"] = re.sub(re_0, "\\4", d["doc_text"])
    
    # discard footnote references

    re_1 = r"(([a-z]|ğ|ü|ş|ı|ö|ç)(\.\d|\d\.)\s)"
    d["doc_text"] = re.sub(re_1, "\\2. ", d["doc_text"])

    # discard page numbers from text
     
    re_2 = r"((^\s?1)|(\s([2-9]|[1-9][0-9]))\s\s)"
    re_3 = r"((^\s?1\s\s)|((\s([2-9]|[1-9][0-9])\s\s)|(\s\s([2-9]|[1-9][0-9])\s)))"
    re_4 = r"((\s([1-9]|[1-9][0-9])\s\s)|(\s\s([2-9]|[1-9][0-9])\s)|(\s" + str(d["no_of_pages"]) + "\s$))"
    result = re.findall(re_2, d["doc_text"])
    if d["no_of_pages"] != len(result):
        result = re.findall(re_3, d["doc_text"])
        re_2 = re_3
    if d["no_of_pages"] != len(res):
        result = re.findall(re_4, d["doc_text"])
        re_2 = re_4

    d["doc_text"] = re.sub(re_2, "", d["doc_text"])

    test_numbers = "".join([r[0].strip()for r in result])
    print(d["no_of_pages"], len(result), d["doc_name"], i, test_numbers)



    # result = re.findall(re_0, d["doc_text"])
    # result_1 = re.findall(re_1, d["doc_text"])

    # print(d["doc_name"], len(result), len(result_1))
    


In [ ]:
test_text = pdf_data[5]["doc_text"]
print(test_text)

In [ ]:
test_text = re.sub(r"((\s){2,}[1-9][0-9]?\s([a-z]|[A-Z]|[0-9]|ğ|ü|ş|ı|ö|ç|Ğ|Ü|Ş|İ|Ö|Ç|:|,|\s|\.|“|”){50,}(\s([2-9]|[1-9][0-9])\s\s))", "\\4", test_text)
print(test_text)

In [ ]:
res = re.findall(r"((\s){2,}[1-9][0-9]?\s([a-z]|[A-Z]|[0-9]|ğ|ü|ş|ı|ö|ç|Ğ|Ü|Ş|İ|Ö|Ç|:|,|\s|\.|“|”){50,}(\s([2-9]|[1-9][0-9])\s\s))", test_text)
print(res)

In [ ]:
test_text = re.sub(r"(([a-z]|ğ|ü|ş|ı|ö|ç)(\.\d|\d\.)\s)", "\\2. ", test_text)
print(test_text)

In [ ]:
for i, d in enumerate(pdf_data):
    
    # discard page numbers from text
     
    re_1 = r"((^\s?1)|(\s([2-9]|[1-9][0-9]))\s\s)"
    re_2 = r"((^\s?1\s\s)|((\s([2-9]|[1-9][0-9])\s\s)|(\s\s([2-9]|[1-9][0-9])\s)))"
    re_3 = r"((\s([1-9]|[1-9][0-9])\s\s)|(\s\s([2-9]|[1-9][0-9])\s)|(\s" + str(d["no_of_pages"]) + "\s$))"
    result = re.findall(re_1, d["doc_text"])
    if d["no_of_pages"] != len(result):
        result = re.findall(re_2, d["doc_text"])
        re_1 = re_2
    if d["no_of_pages"] != len(res):
        result = re.findall(re_3, d["doc_text"])
        re_1 = re_3

    d["doc_text"] = re.sub(re_1, "", d["doc_text"])

    
    

    # if d["no_of_pages"] != len(result):
    #     print(d["no_of_pages"], d["doc_name"], i)

    # test_numbers = "".join([r[0].strip()for r in result])
    # print(d["no_of_pages"], len(result), d["doc_name"], i, test_numbers)

In [ ]:
pdf_data[5]["doc_text"]